# Setup

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import matplotlib.pyplot as plt
import pandas as pd

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [19]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [20]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = True
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-01-20-180945.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=None)                1   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=Ge...                1   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=Ge...                1   
Phragmen(marginal_slate, clear=True, redist=True)                 1   
Phragmen(marginal_slate, clear=True, redist=False)                1   
Phragmen(marginal_slate, clear=False, redist=True)                1   
Phragmen(marginal_slate, clear=False, redist=Fa...                1   
Phragmen(marginal_previous, clear=True, redist=...                1   
Phragmen(marginal_previous, clear=True, redist=...                1   
Phragmen(marginal_previous, clear=False, redist...                1   
Phragmen(marginal_previous, clear=False, redist...                1   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                      1   
ExactTotalUtilityMaximization(utility_transform...                                      1   
LPTotalUtilityMaximization(utility_transform=None)                                      1   
GreedyTotalUtilityMaximization(utility_transfor...                                      1   
ExactTotalUtilityMaximization(utility_transform...                                      1   
LPTotalUtilityMaximization(utility_transform=Ge...                                      1   
GreedyTotalUtilityMaximization(utility_transfor...                                      1   
ExactTotalUtilityMaximization(utility_transform...                                      1   
LPTotalUtilityMaximization(utility_transform=Ge...                                      1   
Phragmen(marginal_slate, clear=True, redist=True)                                       1   
Phragmen(marginal_slate, clear=True, redist=False)                                      1   
Phragmen(marginal_slate, clear=False, redist=True)                                      1   
Phragmen(marginal_slate, clear=False, redist=Fa...                                      1   
Phragmen(marginal_previous, clear=True, redist=...                                      1   
Phragmen(marginal_previous, clear=True, redist=...                                      1   
Phragmen(marginal_previous, clear=False, redist...                                      1   
Phragmen(marginal_previous, clear=False, redist...                                      1   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=None)                            1   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=Ge...                            1   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=Ge...  

In [21]:
# Select subset of rows
# df = df.iloc[3:9]
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=None)                1   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=Ge...                1   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=Ge...                1   
Phragmen(marginal_slate, clear=True, redist=True)                 1   
Phragmen(marginal_slate, clear=True, redist=False)                1   
Phragmen(marginal_slate, clear=False, redist=True)                1   
Phragmen(marginal_slate, clear=False, redist=Fa...                1   
Phragmen(marginal_previous, clear=True, redist=...                1   
Phragmen(marginal_previous, clear=True, redist=...                1   
Phragmen(marginal_previous, clear=False, redist...                1   
Phragmen(marginal_previous, clear=False, redist...                1   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                      1   
ExactTotalUtilityMaximization(utility_transform...                                      1   
LPTotalUtilityMaximization(utility_transform=None)                                      1   
GreedyTotalUtilityMaximization(utility_transfor...                                      1   
ExactTotalUtilityMaximization(utility_transform...                                      1   
LPTotalUtilityMaximization(utility_transform=Ge...                                      1   
GreedyTotalUtilityMaximization(utility_transfor...                                      1   
ExactTotalUtilityMaximization(utility_transform...                                      1   
LPTotalUtilityMaximization(utility_transform=Ge...                                      1   
Phragmen(marginal_slate, clear=True, redist=True)                                       1   
Phragmen(marginal_slate, clear=True, redist=False)                                      1   
Phragmen(marginal_slate, clear=False, redist=True)                                      1   
Phragmen(marginal_slate, clear=False, redist=Fa...                                      1   
Phragmen(marginal_previous, clear=True, redist=...                                      1   
Phragmen(marginal_previous, clear=True, redist=...                                      1   
Phragmen(marginal_previous, clear=False, redist...                                      1   
Phragmen(marginal_previous, clear=False, redist...                                      1   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=None)                            1   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=Ge...                            1   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=Ge...  

## Overall Performance

In [22]:
df.sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                               121
ExactTotalUtilityMaximization(utility_transform=None)                                128
LPTotalUtilityMaximization(utility_transform=None)                                   140
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     138
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))      137
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))         154
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))    133
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))     143
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))        154
Phragmen(marginal_slate, clear=True, redist=True)                                    124
Phragmen(marginal_slate, clear=True, redist=False)                                   124
Phragmen(marginal_sla

## Differential Test Performance

In [23]:
non_uniform_columns = df.loc[:, df.nunique() > 1]
non_uniform_columns


vote                                                       Simple 3  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=None)                1   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=Ge...                1   
GreedyTotalUtilityMaximization(utility_transfor...                1   
ExactTotalUtilityMaximization(utility_transform...                1   
LPTotalUtilityMaximization(utility_transform=Ge...                1   
Phragmen(marginal_slate, clear=True, redist=True)                 0   
Phragmen(marginal_slate, clear=True, redist=False)                0   
Phragmen(marginal_slate, clear=False, redist=True)                0   
Phragmen(marginal_slate, clear=False, redist=Fa...                0   
Phragmen(marginal_previous, clear=True, redist=...                0   
Phragmen(marginal_previous, clear=True, redist=...                0   
Phragmen(marginal_previous, clear=False, redist...                0   
Phragmen(marginal_previous, clear=False, redist...                0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=None)                            1   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=Ge...                            1   
GreedyTotalUtilityMaximization(utility_transfor...                            1   
ExactTotalUtilityMaximization(utility_transform...                            1   
LPTotalUtilityMaximization(utility_transform=Ge...                            1   
Phragmen(marginal_slate, clear=True, redist=True)                             0   
Phragmen(marginal_slate, clear=True, redist=False)                            0   
Phragmen(marginal_slate, clear=False, redist=True)                            0   
Phragmen(marginal_slate, clear=False, redist=Fa...                            0   
Phragmen(marginal_previous, clear=True, redist=...                            0   
Phragmen(marginal_previous, clear=True, redist=...                            0   
Phragmen(marginal_previous, clear=False, redist...                            0   
Phragmen(marginal_previous, clear=False, redist...                            0   

vote                                                                                                    \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                                  1    
ExactTotalUtilityMaximization(utility_transform...                                                  1    
LPTotalUtilityMaximization(utility_transform=None)                                                  1    
GreedyTotalUtilityMaximization(utility_transfor...                                                  1    
ExactTotalUtilityMaximization(utility_transform...                                                  1    
LPTotalUtilityMaximization(utility_transform=Ge...                                                  1    
GreedyTotalUtilityMaximization(utility_transfor...                                                  1    
ExactTotalUtilityMaximization(utility_transform...                                                  1    
LPTotalUtili

These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [24]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)
print("\nCounts for level 1:")
print(level_1_counts)


Counts for level 0:
vote
Simple 3           5
Ex A.1             5
Ex 4.3             5
Ex 1.3             5
Ex 2.1             5
Ex 4.2             5
Ex Alg1.3          5
Ex Alg1.5          5
Ex D.1             5
Ex 1.1 modified    4
Ex Alg A.2         4
Ex B.3             4
Ex 4.1             4
Ex C.2             4
Ex Alg A.1         4
Ex 3.1             1
Ex Alg1.4          1
Ex Alg2.1          1
Name: count, dtype: int64

Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency            16
Non-radical Minimum Utility Pareto Efficiency          16
Maximum Coverage                                       15
Minimum Utility and Total Utility Pareto Efficiency    15
Individual Pareto Efficiency                           10
Name: count, dtype: int64


# Debuggign

In [51]:
case = rated_vote_cases["Ex 1.1 modified"]
alg = ExactTotalUtilityMaximization()
axiom = MinimumAndTotalUtilityParetoAxiom()

In [52]:
df1 = df.loc[str(alg),:]
df1

vote        subtest                                            
Simple 1    Maximum Coverage                                       1
            m-th Happiest Person Pareto Efficiency                 1
            Individual Pareto Efficiency                           1
            Minimum Utility and Total Utility Pareto Efficiency    1
            Non-radical Minimum Utility Pareto Efficiency          1
                                                                  ..
Ex Alg A.2  m-th Happiest Person Pareto Efficiency                 1
            Individual Pareto Efficiency                           1
            Minimum Utility and Total Utility Pareto Efficiency    1
            Non-radical Minimum Utility Pareto Efficiency          1
            Non-radical Total Utility Pareto Efficiency            1
Name: ExactTotalUtilityMaximization(utility_transform=None), Length: 162, dtype: int64

In [53]:
zero_value_columns = df1.loc[df1 == 0]
zero_value_columns


vote             subtest                                            
Ex 1.1 modified  Maximum Coverage                                       0
                 Minimum Utility and Total Utility Pareto Efficiency    0
                 Non-radical Minimum Utility Pareto Efficiency          0
                 Non-radical Total Utility Pareto Efficiency            0
Ex 3.1           Maximum Coverage                                       0
                 Individual Pareto Efficiency                           0
                 Minimum Utility and Total Utility Pareto Efficiency    0
                 Non-radical Minimum Utility Pareto Efficiency          0
                 Non-radical Total Utility Pareto Efficiency            0
Ex 4.1           Non-radical Total Utility Pareto Efficiency            0
Ex 4.2           Maximum Coverage                                       0
                 Individual Pareto Efficiency                           0
                 Minimum Utility and Total 

In [54]:

case.rated_votes

,s1,s2,s3,s4
0,6,2,0,0
1,0,2,0,0
2,0,2,0,0
3,0,0,6,2
4,0,0,0,2
5,0,0,0,2


In [55]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s2', 's4']

In [56]:
axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)

{frozenset({'s2', 's4'})}

In [57]:
num_aug_cases = 24
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases[:num_aug_cases]):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
aug_case_votes.iloc[:,2:]

,axiom_slates,alg_slate,satisfied
0,"{(s2, s4), (s4, s1)}","[s1, s4]",True
1,"{(s3, s2), (s3, s1), (s2, s4)}","[s1, s3]",True
2,"{(s2, s4), (s4, s1)}","[s1, s4]",True
3,"{(s3, s2), (s2, s4)}","[s2, s3]",True
4,"{(s2, s4), (s3, s1)}","[s1, s3]",True
5,"{(s2, s4), (s3, s1)}","[s1, s3]",True
6,"{(s3, s2), (s2, s4)}","[s2, s3]",True
7,"{(s2, s4)}","[s2, s3]",True
8,"{(s2, s4), (s3, s1)}","[s1, s3]",True
9,"{(s2, s4), (s3, s1), (s4, s1)}","[s1, s4]",True


In [58]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied
14,"{(s2, s4)}","[s1, s4]",False
16,"{(s2, s4)}","[s1, s4]",False
19,"{(s2, s4)}","[s1, s4]",False
20,"{(s2, s4)}","[s1, s4]",False
21,"{(s2, s4)}","[s1, s4]",False


In [59]:
f0 = failures.iloc[0]
f0.rated_votes


,s1,s2,s3,s4
0,6.000005e+00,2.000005e+00,0.000005,0.000005
1,2.789860e-06,2.000003e+00,0.000003,0.000003
2,2.615778e-06,2.000003e+00,0.000003,0.000003
3,8.121599e-07,8.121599e-07,6.000001,2.000001
4,1.537782e-06,1.537782e-06,0.000002,2.000002
5,2.410341e-06,2.410341e-06,0.000002,2.000002


In [60]:
f0.assignments

,candidate_id
0,s1
1,s4
2,s4
3,s4
4,s4
5,s4


In [61]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s1', 's4'],
   candidate_id
 0           s1
 1           s4
 2           s4
 3           s4
 4           s4
 5           s4)